## API-kald i Python

### Hvordan bruges API'er?

API'er (til databaser) er meget forskellige, men indeholder typisk de samme følgende grundkomponenter:

**Request:** Ligesom al anden kommunikation med internettet, involverer brug af API at sende en *request* (GET eller POST) til en server. 

**Endpoint:** API'er indeholder typisk flere forskellige *endpoints*. Tænk på disse som "haner", man kan koble sig på, for at udtrække data. Endpoints er URL'er, som en request skal sendes til.

**Parameters:** Parametre er de argumenter eller indstillinger, som endpoint accepterer eller forventer. Nogle er krævet for overhovedet at få data tilbage, andre er valgfrie.

**Authentication:** De fleste API'er kræver en eller anden form for autentificering. Man ser typisk to former for autentificering: HTTPS autentificering (brugernavn og adgangskode) og autentificering via "tokens". Tokens er unikke "nøgler" - en samling af tekst, tegn og tal, som for serveren unikt identificerer, hvem der sender henvendelsen.

## Ekesempel 1: Brug af Statistikbankens API

Link til API dokumentation: https://www.dst.dk/en/Statistik/brug-statistikken/muligheder-i-statistikbanken/api

Man kan tilgå data i Statistikbanken via API.

I det følgende ses nogle eksempler på at bruge API'en via Python.

*Bemærk*: Denne API kræver ikke autentificering

### Udtræk data fra Statistikbankens API

In [ ]:
import requests

statbank_api = "https://api.statbank.dk/v1/data"  #Endpoint of the data API

param = {'table': 'folk1c',
            'format': 'CSV',
            'variables': [{'code': 'OMRÅDE', 'values': ['101', '851']},  #Request in JSON/dictionary
                                                            {'code': 'ALDER', 'values': ['20-24', '25-29']}]
           }

data_req = requests.post(statbank_api, json=param)  #Sending requests

print(data_req.text)  #Printing the raw text output

Data endpointet returnerer komma-separerede værdier som standard (csv).

Dette format er kompatibelt med `pandas` dataframes (`pd.read_csv`):

In [ ]:
from io import StringIO
import pandas as pd

dstdata = StringIO(data_req.text)  #Read the data output as raw text
dstdf = pd.read_csv(dstdata, sep=";")  #Read text as csv
dstdf  #Print data

In [ ]:
dstdf.groupby(['OMRÅDE']).sum()  #Group by municipality and count sum

## Eksempel 2: Brug af Danmarks Adresser Web API (DAWA)

Link til API dokumentation: https://dawadocs.dataforsyningen.dk/dok/api/

Danmarks Adresser Web API (DAWA) kan bruges til at hente data om samt søge efter adresser i Danmark.

*Bemærk:* Denne API kræver ikke autentificering.

In [ ]:
import requests

adress_end = 'https://api.dataforsyningen.dk/adresser' # endpoint til at søge på adresser (API'en har andre endpoints)

# parametre/indstillinger til søgning (se dokumentation)
parameters = {'q': 'Solsort*',  # søg efter adresser der indeholder Solsort
              'kommunekode': '0851'} # søg i kommunekode 0851 (Aalborg)

data_req = requests.get(adress_end, params = parameters)

In [ ]:
data_req.status_code # gik henvendelse igennem?

In [ ]:
data = data_req.json() # data returneres som standard som JSON (tages ud på denne måde)

In [ ]:
from pprint import pprint

pprint(data[:1])

In [ ]:
len(data) # hvor mange hits?

In [ ]:
data[0].keys() # hvilke oplysninger/variable?

DAWA kan sende data tilbage i forskellige formater med parameter "struktur":

In [ ]:
adress_end = 'https://api.dataforsyningen.dk/adresser' # endpoint til at søge på adresser (API'en har andre endpoints)

# parametre/indstillinger til søgning (se dokumentation)
parameters = {'q': 'Solsort*', # søg efter adresser der indeholder Solsort
              'kommunekode': '0851', # søg i kommunekode 0851 (Aalborg)
              'struktur': 'flad'} # send data tilbage i "flad" struktur (data er ikke "nestet" og kan konverteres direkte til tabel)

data_req = requests.get(adress_end, params = parameters)

In [ ]:
data = data_req.json() # data tages stadig ud som JSON, nu er det blot et JSON records format

In [ ]:
data_df = pd.DataFrame(data)

In [ ]:
data_df.head()